In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_d4.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_b5.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_ea.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_a3.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_9a.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_98.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_ee.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_ca.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_88.jpg
/kaggle/input/ident

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
import os

# Set the main directory where all the images are stored
data_dir = '/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset'

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
image_size = (224, 224)
batch_size = 32

In [4]:
data_generator = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet.preprocess_input,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)  # Splitting the data into 80% training and 20% validation

In [5]:
# Load and prepare the dataset
dataset = data_generator.flow_from_directory(data_dir,
                                             target_size=image_size,
                                             batch_size=batch_size,
                                             class_mode='categorical',
                                             subset='training')

validation_dataset = data_generator.flow_from_directory(data_dir,
                                                        target_size=image_size,
                                                        batch_size=batch_size,
                                                        class_mode='categorical',
                                                        subset='validation')

Found 711 images belonging to 8 classes.
Found 174 images belonging to 8 classes.


In [6]:
# Load the ResNet-152 base model
base_model = ResNet152(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

234698864/234698864 [==============================] - 2s 0us/step


In [7]:
# Add custom classification head with regularization
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)  # Dropout layer for regularization
predictions = Dense(dataset.num_classes, activation='softmax', kernel_regularizer=regularizers.l2(0.01))(x)  # L2 regularization

In [8]:
# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

In [9]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
# Train the model
num_train_samples = dataset.samples
num_val_samples = validation_dataset.samples
epochs = 20

history = model.fit(dataset,
                    steps_per_epoch=num_train_samples // batch_size,
                    validation_data=validation_dataset,
                    validation_steps=num_val_samples // batch_size,
                    epochs=epochs)

Epoch 1/20
22/22 [==============================] - 70s 2s/step - loss: 1.6145 - accuracy: 0.4875 - val_loss: 0.9380 - val_accuracy: 0.6938
Epoch 2/20
22/22 [==============================] - 39s 2s/step - loss: 0.9471 - accuracy: 0.6804 - val_loss: 0.8480 - val_accuracy: 0.7312
Epoch 3/20
22/22 [==============================] - 42s 2s/step - loss: 0.7261 - accuracy: 0.7747 - val_loss: 0.6809 - val_accuracy: 0.8313
Epoch 4/20
22/22 [==============================] - 42s 2s/step - loss: 0.6710 - accuracy: 0.7614 - val_loss: 0.6921 - val_accuracy: 0.7812
Epoch 5/20
22/22 [==============================] - 39s 2s/step - loss: 0.5349 - accuracy: 0.8409 - val_loss: 0.6841 - val_accuracy: 0.7688
Epoch 6/20
22/22 [==============================] - 38s 2s/step - loss: 0.5344 - accuracy: 0.8306 - val_loss: 0.6946 - val_accuracy: 0.7937
Epoch 7/20
22/22 [==============================] - 39s 2s/step - loss: 0.5453 - accuracy: 0.8262 - val_loss: 0.8743 - val_accuracy: 0.7125
Epoch 8/20
22/22 [==

In [11]:
test_loss, test_accuracy = model.evaluate(validation_dataset, steps=num_val_samples // batch_size)
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

5/5 [==============================] - 7s 1s/step - loss: 0.6763 - accuracy: 0.7688
Test loss: 0.6763175129890442
Test accuracy: 0.768750011920929


In [12]:
train_loss = history.history['loss'][-1]
train_accuracy = history.history['accuracy'][-1]
val_loss = history.history['val_loss'][-1]
val_accuracy = history.history['val_accuracy'][-1]

In [13]:
print('Training loss:', train_loss)
print('Training accuracy:', train_accuracy)
print('Validation loss:', val_loss)
print('Validation accuracy:', val_accuracy)

Training loss: 0.2600851356983185
Training accuracy: 0.9307805299758911
Validation loss: 0.6663914918899536
Validation accuracy: 0.762499988079071
